See the parallel axis theorem to calculate the inertia of a combination of solids

In [2]:
# Class Solid
from stl import mesh
import numpy as np
from mpl_toolkits import mplot3d
from matplotlib import pyplot
import plotly.graph_objects as go


class Solid:
    '''
    Class representing a solid object containing its mesh, and its mass properties (volume, mass, COG, matrix of inertia)

    Attributes:
        brut_mesh (BruteMesh): Object containing 'veritices'(numpy.array) and 'faces'(numpy.array) of the solid's mesh
        stl_mesh (stl.mesh): The corresponding stl.mesh object for the solid.
        plot_brute_mesh (BruteMesh): Object containing 'veritices'(numpy.array) and 'faces'(numpy.array) of the solid's to plot (may be different from the solid mesh)
        plot_stl_mesh (stl.mesh): The corresponding stl.mesh.
    '''


    class BruteMesh:
        '''
        Class used to store the brute mesh of the Solid.
        '''
        def __init__(self, vertices, faces) -> None:
            self.vertices = vertices
            self.faces = faces


    def __init__(self, brute_mesh : BruteMesh, stl_mesh : mesh, plot_brute_mesh : BruteMesh, plot_stl_mesh : mesh) -> None:
        self.brute_mesh = brute_mesh
        self.stl_mesh = stl_mesh
        self.plot_brute_mesh = plot_brute_mesh
        self.plot_stl_mesh = plot_stl_mesh


    def combine(self, solid):
        '''
        Combine two Solid objects.
        
        Returns:
            A Solid containing the combination of the two input Solid.
        '''
        # Normal properties of the Solid
        v1,f1, vect1 = self.brute_mesh.vertices, self.brute_mesh.faces, self.stl_mesh.vectors
        v2,f2, vect2 = solid.brute_mesh.vertices, solid.brute_mesh.faces, solid.stl_mesh.vectors

        vertices = np.concatenate((v1,v2))
        f2 += len(v1)
        faces = np.concatenate((f1,f2))

        brute_mesh = self.BruteMesh(vertices, faces)

        vectors = np.concatenate((vect1, vect2))
        data = np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype)
        data['vectors'] = vectors

        stl_mesh = mesh.Mesh(data)

        # Plotting properties of the Solid
        pv1,pf1, pvect1 = self.plot_brute_mesh.vertices, self.plot_brute_mesh.faces, self.plot_stl_mesh.vectors
        pv2,pf2, pvect2 = solid.plot_brute_mesh.vertices, solid.plot_brute_mesh.faces, solid.plot_stl_mesh.vectors
        
        pvertices = np.concatenate((pv1,pv2))
        pf2 += len(pv1)
        pfaces = np.concatenate((pf1,pf2))

        plot_brute_mesh = self.BruteMesh(pvertices, pfaces)

        pvectors = np.concatenate((pvect1, pvect2))
        pdata = np.zeros(pfaces.shape[0], dtype=mesh.Mesh.dtype)
        pdata['vectors'] = pvectors

        plot_stl_mesh = mesh.Mesh(pdata)

        return Solid(brute_mesh=brute_mesh, stl_mesh=stl_mesh, plot_brute_mesh=plot_brute_mesh, plot_stl_mesh=plot_stl_mesh)
    

    def show_mpl(self):
        '''
        Plot the Solid using matplotlib.
        '''
        pyplot.close()
        # Create a new plot
        figure = pyplot.figure()
        axes = figure.add_subplot(projection='3d')
        # Add mesh to the plot
        axes.add_collection3d(mplot3d.art3d.Poly3DCollection(self.plot_stl_mesh.vectors))
        # Auto scale to the mesh size
        scale = self.plot_stl_mesh.points.flatten()
        axes.auto_scale_xyz(scale, scale, scale)
        # Show the plot to the screen
        pyplot.show()
        
        return figure

        
    def show_go(self):
        '''
        Plot the Solid using plotly.graph_objects.
        '''
        faces = self.plot_brute_mesh.faces
        vertices = self.plot_brute_mesh.vertices

        x,y,z = vertices.T
        i,j,k = faces.T
        figure = go.Figure(data=[go.Mesh3d(x=x,y=y,z=z,i=i,j=j,k=k,showscale=True)])

        # Set the aspect ratio of the 3D scene to match the data
        figure.update_layout(scene=dict(aspectmode="data"))

        figure.show()

    def show_all_mpl(self):
        '''
        Plot the Solid using matplotlib.
        '''
        pyplot.close()
        # Create a new plot
        figure = pyplot.figure()
        axes = figure.add_subplot(projection='3d')
        # Add mesh to the plot
        axes.add_collection3d(mplot3d.art3d.Poly3DCollection(self.stl_mesh.vectors))
        # Auto scale to the mesh size
        scale = self.stl_mesh.points.flatten()
        axes.auto_scale_xyz(scale, scale, scale)
        # Show the plot to the screen
        pyplot.show()
        
        return figure

        
    def show_all_go(self):
        '''
        Plot the Solid using plotly.graph_objects.
        '''
        faces = self.brute_mesh.faces
        vertices = self.brute_mesh.vertices

        x,y,z = vertices.T
        i,j,k = faces.T
        figure = go.Figure(data=[go.Mesh3d(x=x,y=y,z=z,i=i,j=j,k=k,showscale=True)])

        # Set the aspect ratio of the 3D scene to match the data
        figure.update_layout(scene=dict(aspectmode="data"))

        figure.show()

In [3]:
# Util - Translate and normalize
import numpy as np

# Function used to translate vertices of a mesh
def translate(vertices, trans_vect  = np.zeros(3)):
    return vertices + trans_vect

# Function used to normalize a list of vector
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

In [4]:
# Nose equations
class Nose:
    '''
    Object containing the points for the rocket's nose.

    Attributes:
        out_points (numpy.array): Points of the outer side of the nose.
        in_points (numpy.array): Points of the inner side of the nose.
    '''
    def __init__(self, out_points, in_points) -> None:
        self.out_points = out_points
        self.in_points = in_points

def create_nose(x, y_out, y_prime, L, thick):
    '''
    Create a Nose object given the basic coordinates of the points of a nose shape.

    Args:
        x (numpy.array): x-coordinates of points on the nose.
        y_out (numpy.array): y-coordinates of points on the outer side of the nose.
        y_prime (numpy.array): derivatives of the shape along the x-axis (used to translate the outer side to create the inner side of the nose).
        L (float): Length of the nose
        thick (float): Thickness of the nose.

    Returns:
        A Nose object containing 3D vectors corresponding to both outer and inner side of the nose.
    '''
    n = len(y_out)
    out_points = np.array([x,y_out]).T
    in_points = out_points + thick * normalized(np.array([y_prime, -np.ones(n)]).T)

    first = -1
    while in_points[first+1, 1]<=0:
        first+=1
    in_points = in_points[first:]
    in_points = in_points[in_points[:,0]<L]
    last = first+len(in_points)-1

    # Add the first point (y = 0) and the last point (x = L)
    if len(in_points)>=2:
        # First point
        # This special case is for elliptic,haack and power series noses which do not have finite derivative when x=0
        if y_prime[0]==-1:
            first_point = np.array([thick, 0])
        else: # General case
            x1,y1 = in_points[0,0], in_points[0,1]
            x2,y2 = in_points[1,0], in_points[1,1]
            xvalue = x1 - y1*(x2-x1)/(y2-y1)
            first_point = np.array([xvalue, 0])
        in_points[0] = first_point

        # Last point
        x1,y1,y_p = in_points[-1,0], in_points[-1,1], y_prime[last]
        yl = y_p*(L-x1)+y1
        # To avoid the last point from crossing over outer points
        last_point = np.array([[L,yl]]) if yl < out_points[-1,1]-thick/2 else np.array([[L, out_points[-1,1]-thick]])
        in_points = np.concatenate((in_points, last_point))
        
    out_points = np.pad(out_points, [(0,0),(0,1)], mode = 'constant')
    in_points = np.pad(in_points, [(0,0),(0,1)], mode = 'constant')

    return Nose(out_points=out_points, in_points=in_points)

# Different equations for nose shape (all nose equation equal 0 at the origin)

def tangent_ogive(x, R, L, thick=.01, C=0):
    rho = (R**2 + L**2)/(2*R)
    y_out = np.sqrt(rho**2 - (L-x)**2) + R - rho 
    # Calculate the derivative of the function
    y_prime = (L-x)/np.sqrt(rho**2 - (L-x)**2)

    return create_nose(x,y_out,y_prime,L,thick)

def ellipse(x,R,L, thick=.01, C=0):
    y_out = R*np.sqrt(1-((L-x[1:])/L)**2)
    y_out = np.concatenate((np.array([0.]), y_out))
    # Calculate the derivative of the function
    y_prime = R/L**2*(L-x[1:])/np.sqrt(1-((L-x[1:])/L)**2)
    y_prime = np.pad(y_prime, (1,0), mode='constant', constant_values=-1)

    return create_nose(x,y_out,y_prime,L,thick)

def parabole(x, R, L, C=0., thick=.01):
    # C in [0,1]
    y_out = R*((2*(x/L)-C*(x/L)**2)/(2-C))
    # Calculate the derivative of the function
    y_prime = 2*R/(L*(2-C))*(1-C*x)

    return create_nose(x,y_out,y_prime,L,thick)

def power_series(x, R, L, C=0., thick=.01):
    # C in [0,1)
    y_out = R*(x/L)**C
    y_prime = R*C*(x[1:]/L)**(C-1)
    y_prime = np.pad(y_prime, (1,0), mode='constant', constant_values=-1)

    return create_nose(x,y_out,y_prime,L,thick)

def haack_series(x, R, L, C=0., thick=.01):
    # If C = 0, LD-Haack (Von Kármán), if C=1/3, LV-Haack, minimum drag, if C=2/3, tangent to the body at the base
    theta = np.arccos(1-2*x/L)    
    y_out = R/np.sqrt(np.pi) * np.sqrt(theta- np.sin(2*theta)/2 + C*np.sin(theta)**3)
    # Calculate the derivative of the function
    xL = x*(L-x)/L**2
    A = 2*R*((3*C+2)*L-6*C*x)*np.sqrt(2/np.pi)/L**2
    B = np.sqrt(xL)
    C = np.sqrt(16*C*xL**1.5 + np.sin(2*theta)+2*theta)
    C[0]=1
    y_prime = A*B/C

    return create_nose(x,y_out,y_prime,L,thick)

def cone(x, R, L, thick=.01, C=0):
    y_out = x*R/L
    # Calculate the derivative of the function
    y_prime = R/L*np.ones(len(x))

    return create_nose(x,y_out,y_prime,L,thick)

In [6]:
# Mesh creation
import numpy as np
from stl import mesh
from scipy.spatial.transform import Rotation as R


# Functions to manually create meshes for cylinder, fins and nose of the rocket given its caracteristics

def half_cylinder(outer_radius = .5, inner_radius = .49, height = 2, pnum = 25, pos = 0.):
    '''
    Create plotting part of the empty body of the rocket (i.e. a half cylinder).

    Args:
        outer_radius (float): Outer radius of the rocket.
        inner_radius (float): Inner radius of the rocket.
        height (float): The height of the rocket.
        pnum (int): number of points around the cylinder.
        pos (float): position of cylinder's bottom.

    Returns:
        The plotting mesh for the rocket's body.
    '''
    theta = np.linspace(0, np.pi, pnum, endpoint=True)

    cos, sin = np.cos(theta), np.sin(theta)

    y_out = outer_radius * cos
    z_out = outer_radius * sin
    y_in = inner_radius * cos
    z_in = inner_radius * sin

    x_bottom = pos + np.zeros(pnum)
    x_top = (height+pos) * np.ones(pnum)

    vertices = np.concatenate((np.array([x_bottom, y_out, z_out]).T,
                               np.array([x_top, y_out, z_out]).T,
                               np.array([x_bottom, y_in, z_in]).T,
                               np.array([x_top, y_in, z_in]).T))

    faces = []
    
    # Cylinder
    for i in range(pnum-1):
        # Outer one
        faces.append([i, i+1, pnum+i+1])
        faces.append([pnum+i+1, pnum+i, i])
        # Inner one
        faces.append([3*pnum+i+1, 2*pnum+i+1, 2*pnum+i])
        faces.append([2*pnum+i, 3*pnum+i, 3*pnum+i+1])

    # Bottom face
    for i in range(pnum-1):
        faces.append([i,2*pnum+i,i+1])
        faces.append([i+1, 2*pnum+i, 2*pnum+i+1])
    # Top face
    for i in range(pnum-1):
        faces.append([pnum+i, pnum+i+1, 3*pnum+i])
        faces.append([pnum+i+1, 3*pnum+i+1, 3*pnum+i])

    faces.append([0, pnum, 3*pnum])
    faces.append([0, 3*pnum, 2*pnum])
    faces.append([3*pnum-1, 4*pnum-1, 2*pnum-1])
    faces.append([3*pnum-1, 2*pnum-1, pnum-1])

    faces = np.array(faces)

    brute_mesh = Solid.BruteMesh(vertices=vertices, faces=faces)

    cylinder_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            cylinder_mesh.vectors[i][j] = vertices[f[j], :]

    return brute_mesh, cylinder_mesh


def empty_cylinder(outer_radius = .5, inner_radius = .49, height = 2, pnum = 50, pos = 0.):
    '''
    Create a Solid object representing the empty body of the rocket.

    Args:
        outer_radius (float): Outer radius of the rocket.
        inner_radius (float): Inner radius of the rocket.
        height (float): The height of the rocket.
        pnum (int): number of points around the cylinder.
        pos (float): position of cylinder's bottom.

    Returns:
        A Solid object containing all data necessary to manipulate, plot and calculate properties of the cylinder.
    '''
    theta = np.linspace(0, 2*np.pi, pnum, endpoint=False)

    cos, sin = np.cos(theta), np.sin(theta)

    y_out = outer_radius * cos
    z_out = outer_radius * sin
    y_in = inner_radius * cos
    z_in = inner_radius * sin

    x_bottom = pos + np.zeros(pnum)
    x_top = (height+pos) * np.ones(pnum)

    vertices = np.concatenate((np.array([x_bottom, y_out, z_out]).T,
                               np.array([x_top, y_out, z_out]).T,
                               np.array([x_bottom, y_in, z_in]).T,
                               np.array([x_top, y_in, z_in]).T))

    faces = []
    
    # Cylinder
    for i in range(pnum-1):
        # Outer one
        faces.append([i, i+1, pnum+i+1])
        faces.append([pnum+i+1, pnum+i, i])
        # Inner one
        faces.append([3*pnum+i+1, 2*pnum+i+1, 2*pnum+i])
        faces.append([2*pnum+i, 3*pnum+i, 3*pnum+i+1])
    # Last outer triangles
    faces.append([pnum-1, 0, pnum])
    faces.append([pnum, 2*pnum-1, pnum-1])
    # Last inner triangles
    faces.append([3*pnum, 2*pnum, 3*pnum-1])
    faces.append([3*pnum-1, 4*pnum-1, 3*pnum])

    # Bottom face
    for i in range(pnum-1):
        faces.append([i,2*pnum+i,i+1])
        faces.append([i+1, 2*pnum+i, 2*pnum+i+1])
    faces.append([pnum-1, 3*pnum-1, 0])
    faces.append([0, 3*pnum-1, 2*pnum])
    # Top face
    for i in range(pnum-1):
        faces.append([pnum+i, pnum+i+1, 3*pnum+i])
        faces.append([pnum+i+1, 3*pnum+i+1, 3*pnum+i])
    faces.append([2*pnum-1, pnum, 4*pnum-1])
    faces.append([pnum, 3*pnum, 4*pnum-1])
   
    faces = np.array(faces)

    brute_mesh = Solid.BruteMesh(vertices=vertices, faces=faces)

    cylinder_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            cylinder_mesh.vectors[i][j] = vertices[f[j], :]

    plot_brute_mesh, plot_cylinder_mesh = half_cylinder(outer_radius=outer_radius, inner_radius=inner_radius, height=height, pnum=pnum//2, pos=pos)

    return Solid(brute_mesh=brute_mesh, stl_mesh=cylinder_mesh, plot_brute_mesh=plot_brute_mesh, plot_stl_mesh=plot_cylinder_mesh)


def cylinder(radius = .5, height = 2, pnum = 50, pos = 0.):
    '''
    Create a Solid object representing a cylindric distribution of mass in the rocket.

    Args:
        radius (float): Outer radius of the cylinder.
        height (float): The height of the cylinder.
        pnum (int): number of points around the cylinder.
        pos (float): position of cylinder's bottom.

    Returns:
        A Solid object containing all data necessary to manipulate, plot and calculate properties of the cylinder.
    '''
    theta = np.linspace(0, 2*np.pi, pnum, endpoint=False)
    y = radius * np.cos(theta)
    z = radius * np.sin(theta)

    x_bottom = pos + np.zeros(pnum)
    x_top = (height+pos) * np.ones(pnum)

    vertices = np.concatenate((np.array([x_bottom, y, z]).T, np.array([x_top, y, z]).T, np.array([[pos,0,0]]), np.array([[height+pos,0,0]])))

    faces = []

    end = len(vertices)
    
    # Bottom face
    for i in range(pnum-1):
        faces.append([end-2,i+1,i])
    faces.append([end-2,0,pnum-1])
    # Top face
    for i in range(pnum-1):
        faces.append([end-1,pnum+i,pnum+i+1])
    faces.append([end-1,2*pnum-1,pnum])
    # Cylinder
    for i in range(pnum-1):
        faces.append([i, i+1, pnum+i+1])
        faces.append([pnum+i+1, pnum+i, i])
    faces.append([pnum-1, 0, pnum])
    faces.append([pnum, 2*pnum-1, pnum-1])

    faces = np.array(faces)

    brute_mesh = Solid.BruteMesh(vertices=vertices, faces=faces)

    cylinder_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            cylinder_mesh.vectors[i][j] = vertices[f[j], :]

    return Solid(brute_mesh=brute_mesh, stl_mesh=cylinder_mesh, plot_brute_mesh=brute_mesh, plot_stl_mesh=cylinder_mesh)


def fins(Cr=.2,Ct=.1, Xt=.1, s=.15, thick=.01,fnum=4, radius=.5, pos=0.):
    '''
    Create a Solid object containing the mesh for the fins of the rocket and their various properties (see class Solid).
    See fin.png for the definition of Cr,...,s.

    Args:
        thick (float): Thickness of the fin.
        fnum (int): The number of fins.
        radius (float): The radius of the rocket.
        pos (float): the position of the bottom of the fins (0 is the position of the bottom of the rocket).

    Returns:
        A Solid object containing all data necessary to manipulate, plot and calculate properties of the fins.
    '''
    # Mesh for one fin
    vertices = np.array([[Cr, 0, -thick/2],
                         [Cr, 0, thick/2],
                         [Cr-Xt, s, thick/2],
                         [Cr-Xt, s, -thick/2],
                         [0, 0, -thick/2],
                         [0, 0, thick/2],
                         [Cr-Xt-Ct, s, thick/2],
                         [Cr-Xt-Ct, s, -thick/2]])
    faces = np.array([[0, 3, 1],
                      [2, 1, 3],
                      [0, 1, 4],
                      [1, 5, 4],
                      [4, 5, 6],
                      [4, 6, 7],
                      [2, 3, 6],
                      [3, 7, 6],
                      [1, 2, 5],
                      [2, 6, 5],
                      [0, 4, 3],
                      [3, 4, 7]])
    
    vertices = translate(vertices=vertices, trans_vect=np.array([pos, radius, 0]))

    total_vertices = vertices
    total_faces = faces

    n_vertice = len(vertices)
    n_face = len(faces)

    # Total mesh
    for n in range(1,fnum):
        # rotation and translation of the fin according to its position
        angle = 2*np.pi*n/fnum
        rotation = R.from_euler('xyz', angles=[angle,0,0], degrees=False)

        new_vertices = rotation.apply(vertices)
        new_faces = faces + len(total_vertices)

        total_vertices = np.concatenate((total_vertices, new_vertices))
        total_faces = np.concatenate((total_faces, new_faces))

    # Plotting mesh consisting in the half of the toal mesh
    num = int(np.ceil((fnum+1)/2))

    plot_total_vertices = total_vertices[:n_vertice*num]
    plot_total_faces = total_faces[:n_face*num]
    
    brute_mesh = Solid.BruteMesh(vertices=total_vertices, faces=total_faces)
    plot_brute_mesh = Solid.BruteMesh(vertices=plot_total_vertices, faces=plot_total_faces)

    fins = mesh.Mesh(np.zeros(total_faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(total_faces):
        for j in range(3):
            fins.vectors[i][j] = total_vertices[f[j],:]

    data = np.zeros(plot_total_faces.shape[0], dtype=mesh.Mesh.dtype)
    data['vectors'] = fins.vectors[:n_face*num,:]

    plot_fins = mesh.Mesh(data)

    return Solid(brute_mesh=brute_mesh, stl_mesh=fins, plot_brute_mesh=plot_brute_mesh, plot_stl_mesh=plot_fins)


def half_nose(nose_obj, Cpnum=10):
    '''
    Create plotting part of the nose of the rocket.

    Args:
        shape_function (string): The function giving the nose's shape (see code for available functions).
        radius (float): The radius of the nose (typically similar to rocket's radius).
        length (float): The length of the nose.
        curve_param (number): Curve parameter used in the shape_function (see the definition of each shape_function for more detail).
        thick (float): Thickness of the nose.
        pos (float): Position of nose bottom with respect to rocket's bottom.
        Vpnum (int): Number of points on the curve.
        Cpnum (int): Number of points around the nose.
        
    Returns:
        The plotting mesh for the nose.
    '''
    out_vertices = nose_obj.out_points
    in_vertices = nose_obj.in_points
    out_vertices_to_rotate = out_vertices[1:]
    in_vertices_to_rotate = in_vertices[1:]

    OVpnum = len(out_vertices)
    IVpnum = len(in_vertices)

    out_faces = []
    in_faces = []

    for j in range(1,Cpnum):
        # Rotation and translation of the fin according to its position
        angle = np.pi*j/(Cpnum-1)
        rotation = R.from_euler('xyz', angles=[angle,0,0], degrees=False)

        # Rotate and concatenate
        out_vertices = np.concatenate((out_vertices, rotation.apply(out_vertices_to_rotate)))
        in_vertices = np.concatenate((in_vertices, rotation.apply(in_vertices_to_rotate)))

        # Making the triangles on the tip of the nose
        if j == 1:
            # Outer first column of triangles
            out_faces.append([OVpnum,1,0])
            for i in range(1,OVpnum-1):
                out_faces.append([OVpnum+i-1, OVpnum+i, i])
                out_faces.append([OVpnum+i,i+1,i])
            
            # Inner first column of trianges
            in_faces.append([0,1,IVpnum])
            for i in range(1,IVpnum-1):
                in_faces.append([i,IVpnum+i,IVpnum+i-1])
                in_faces.append([i,i+1,IVpnum+i])

        else:
            # Remaining faces on the outer side
            for i in range(OVpnum-2):
                if (i==0):
                    # Single top triangle
                    out_faces.append([OVpnum + (OVpnum-1)*(j-1),OVpnum + (OVpnum-1)*(j-2),0])
                    # first rectangle
                    left = OVpnum + (OVpnum-1)*(j-2)
                    right = OVpnum + (OVpnum-1)*(j-1)
                    out_faces.append([right, right+1 ,left])
                    out_faces.append([right+1, left+1 ,left])
                else:
                    # other rectangles
                    left = i + OVpnum + (OVpnum-1)*(j-2)
                    right = i + OVpnum + (OVpnum-1)*(j-1)
                    out_faces.append([right, right+1 ,left])
                    out_faces.append([right+1, left+1 ,left])
            # Remaining faces on the inner side
            for i in range(IVpnum-2):
                if (i==0):
                    # Single top triangle
                    in_faces.append([0,IVpnum + (IVpnum-1)*(j-2),IVpnum + (IVpnum-1)*(j-1)])
                    # first rectangle
                    left = IVpnum + (IVpnum-1)*(j-2)
                    right = IVpnum + (IVpnum-1)*(j-1)
                    in_faces.append([left,right+1,right])
                    in_faces.append([left,left+1,right+1])
                else:
                    # other rectangles
                    left = i + IVpnum + (IVpnum-1)*(j-2)
                    right = i + IVpnum + (IVpnum-1)*(j-1)
                    in_faces.append([left,right+1,right])
                    in_faces.append([left,left+1,right+1])

    nout = len(out_vertices)

    out_faces = np.array(out_faces)
    in_faces = np.array(in_faces) + nout

    # Closing the nose
    close_faces = []
    for i in range(1,Cpnum):
        top_r, top_l = i*(OVpnum-1), (i+1)*(OVpnum-1)
        bot_r, bot_l = i*(IVpnum-1)+nout, (i+1)*(IVpnum-1)+nout
        close_faces.append([top_r, top_l, bot_r])
        close_faces.append([top_l, bot_l, bot_r])

    # Closing the lateral side
    out_end = (Cpnum-2)*(OVpnum-1)+OVpnum
    in_end = nout + (Cpnum-2)*(IVpnum-1)+IVpnum
    # Top first triangles
    close_faces.append([1,nout,0])
    close_faces.append([nout,out_end,0])
    n_tri = OVpnum-IVpnum
    for i in range(1,n_tri):
        close_faces([i,i+1,nout])
        close_faces.append([out_end+1,out_end,nout])
        out_end+=1
    # First rectangle
    close_faces.append([n_tri,nout+1,nout])
    close_faces.append([n_tri,n_tri+1,nout+1])
    close_faces.append([out_end+1,out_end,nout])
    close_faces.append([nout,in_end,out_end+1])
    # Remaining rectangles
    out_end-=1
    for i in range(1,IVpnum-1):
        close_faces.append([n_tri+i,nout+i+1,nout+i])
        close_faces.append([n_tri+i,n_tri+i+1,nout+i+1])
        close_faces.append([out_end+i+1, out_end+i, in_end+i-1])
        close_faces.append([in_end+i-1, in_end+i ,out_end+i+1])

    close_faces = np.array(close_faces)
    
    vertices = np.concatenate((out_vertices, in_vertices))
    faces = np.concatenate((out_faces, in_faces, close_faces))

    brute_mesh = Solid.BruteMesh(vertices=vertices, faces=faces)

    stl_nose = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            stl_nose.vectors[i][j] = vertices[f[j],:]

    return brute_mesh, stl_nose


def nose(shape_function = 'cone', radius=.5, length = 1, curve_param = 0, thick = .1, pos=0, OVpnum = 20, Cpnum=20):
    '''
    Create a Solid representing the nose of the rocket.

    Args:
        shape_function (string): The function giving the nose's shape (see code for available functions).
        radius (float): The radius of the nose (typically similar to rocket's radius).
        length (float): The length of the nose.
        curve_param (number): Curve parameter used in the shape_function (see the definition of each shape_function for more detail).
        thick (float): Thickness of the nose.
        pos (float): Position of nose bottom with respect to rocket's bottom.
        Vpnum (int): Number of points on the curve.
        Cpnum (int): Number of points around the nose.
        
    Returns:
        A Solid object containing all data necessary to manipulate, plot and calculate properties of the nose.
    '''

    valid_functions = ['tangent_ogive','ellipse','parabole','power_series','haack_series','cone']  # predefined list of valid strings
    assert shape_function in valid_functions, f"Invalid shape function. Valid options are: {', '.join(valid_functions)}"

    # function = getattr(__main__<<the module in which the function is defined, shape_function) Use theses lines for futur code
    # function(x, radius, length, C=curve_param, thick=thick)

    x = np.linspace(0, length, OVpnum, endpoint=True)
    nose_points = eval(shape_function + "(x, radius, length, C=curve_param, thick=thick)")

    out_vertices = nose_points.out_points
    in_vertices = nose_points.in_points

    rotation_xz = R.from_euler('xyz',angles=[np.pi,0,np.pi], degrees=False)
    transvect = np.array([pos+length, 0, 0])
    out_vertices = translate(rotation_xz.apply(out_vertices), trans_vect  = transvect)
    in_vertices = translate(rotation_xz.apply(in_vertices), trans_vect  = transvect)

    nose_points = Nose(out_vertices,in_vertices)

    out_vertices_to_rotate = out_vertices[1:]
    in_vertices_to_rotate = in_vertices[1:]

    OVpnum = len(out_vertices)
    IVpnum = len(in_vertices)

    out_faces = []
    in_faces = []

    for j in range(1,Cpnum):
        # Rotation and translation of the fin according to its position
        angle = 2*np.pi*j/Cpnum
        rotation = R.from_euler('xyz', angles=[angle,0,0], degrees=False)

        # Rotate and concatenate
        out_vertices = np.concatenate((out_vertices, rotation.apply(out_vertices_to_rotate)))
        in_vertices = np.concatenate((in_vertices, rotation.apply(in_vertices_to_rotate)))

        # Making the triangles on the tip of the nose
        if j == 1:
            # Outer first column of triangles
            out_faces.append([OVpnum,1,0])
            for i in range(1,OVpnum-1):
                out_faces.append([OVpnum+i-1, OVpnum+i, i])
                out_faces.append([OVpnum+i,i+1,i])
            
            # Inner first column of trianges
            in_faces.append([0,1,IVpnum])
            for i in range(1,IVpnum-1):
                in_faces.append([i,IVpnum+i,IVpnum+i-1])
                in_faces.append([i,i+1,IVpnum+i])

        else:
            # Remaining faces on the outer side
            for i in range(OVpnum-2):
                if (i==0):
                    # Single top triangle
                    out_faces.append([OVpnum + (OVpnum-1)*(j-1),OVpnum + (OVpnum-1)*(j-2),0])
                    # first rectangle
                    left = OVpnum + (OVpnum-1)*(j-2)
                    right = OVpnum + (OVpnum-1)*(j-1)
                    out_faces.append([right, right+1 ,left])
                    out_faces.append([right+1, left+1 ,left])
                else:
                    # other rectangles
                    left = i + OVpnum + (OVpnum-1)*(j-2)
                    right = i + OVpnum + (OVpnum-1)*(j-1)
                    out_faces.append([right, right+1 ,left])
                    out_faces.append([right+1, left+1 ,left])
            # Remaining faces on the inner side
            for i in range(IVpnum-2):
                if (i==0):
                    # Single top triangle
                    in_faces.append([0,IVpnum + (IVpnum-1)*(j-2),IVpnum + (IVpnum-1)*(j-1)])
                    # first rectangle
                    left = IVpnum + (IVpnum-1)*(j-2)
                    right = IVpnum + (IVpnum-1)*(j-1)
                    in_faces.append([left,right+1,right])
                    in_faces.append([left,left+1,right+1])
                else:
                    # other rectangles
                    left = i + IVpnum + (IVpnum-1)*(j-2)
                    right = i + IVpnum + (IVpnum-1)*(j-1)
                    in_faces.append([left,right+1,right])
                    in_faces.append([left,left+1,right+1])

    # Last column of outer side
    for i in range(OVpnum-1):
        end = OVpnum + (OVpnum-1)*(Cpnum-2)
        if i == 0:
            out_faces.append([0,1,end])
        else:
            out_faces.append([i,end+i,end+i-1])
            out_faces.append([i,i+1,end+i])
    # Last column of inner side
    for i in range(IVpnum-1):
        end = IVpnum + (IVpnum-1)*(Cpnum-2)
        if i == 0:
            in_faces.append([end,1,0])
        else:
            in_faces.append([end+i-1,end+i,i])
            in_faces.append([end+i,i+1,i])

    nout = len(out_vertices)
    nin = len(in_vertices)

    out_faces = np.array(out_faces)
    in_faces = np.array(in_faces) + nout

    # Closing the nose
    close_faces = []
    for i in range(1,Cpnum):
        top_r, top_l = i*(OVpnum-1), (i+1)*(OVpnum-1)
        bot_r, bot_l = i*(IVpnum-1)+nout, (i+1)*(IVpnum-1)+nout
        close_faces.append([top_r, top_l, bot_r])
        close_faces.append([top_l, bot_l, bot_r])
    top_r, top_l = nout-1, OVpnum-1
    bot_r, bot_l = nin-1+nout, (IVpnum-1)+nout
    close_faces.append([top_r, top_l, bot_r])
    close_faces.append([top_l, bot_l, bot_r])

    close_faces = np.array(close_faces)

    vertices = np.concatenate((out_vertices, in_vertices))
    faces = np.concatenate((out_faces, in_faces, close_faces))

    brute_mesh = Solid.BruteMesh(vertices=vertices, faces=faces)

    nose = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            nose.vectors[i][j] = vertices[f[j],:]

    plot_brute_mesh, plot_nose = half_nose(nose_points, Cpnum = Cpnum//2)

    return Solid(brute_mesh=brute_mesh, stl_mesh=nose, plot_brute_mesh=plot_brute_mesh, plot_stl_mesh=plot_nose)



In [7]:
%matplotlib widget 

pyplot.close()

my_nose = nose(shape_function = 'tangent_ogive',thick=0.1)
fig = my_nose.show_go()

In [8]:


radius = 0.07
thick = 0.01
height = 2
Cr = .25
Xt = .15
Ct = .15
s = 0.2
fnum = 6

my_nose = nose(shape_function = 'tangent_ogive',thick=thick, pos=height, radius=radius, length=0.5)
fin = fins(Ct = Ct, Cr = Cr, s = s, Xt=Xt, radius=radius, fnum=fnum)
body = empty_cylinder(outer_radius = radius, inner_radius=radius - thick, height=height)

rocket = fin.combine(body).combine(my_nose)

rocket.show_go()

dens = 1e3
volume, vmass, cog, inertia = rocket.stl_mesh.get_mass_properties_with_density(dens)
print("Density (kg/m^3)                        = {0}".format(dens))
print("Volume (m^3)                            = {0}".format(volume))
print("Mass   (kg)                             = {0}".format(vmass))
print("Position of the center of gravity (COG) = {0}".format(cog))
print("Inertia matrix at expressed at the COG  = {0}".format(inertia[0,:]))
print("                                          {0}".format(inertia[1,:]))
print("                                          {0}".format(inertia[2,:]))

Density (kg/m^3)                        = 1000.0
Volume (m^3)                            = 0.009244048907930846
Mass   (kg)                             = 9.244048907930846
Position of the center of gravity (COG) = [ 5.94342785e-01  2.27792283e-12 -5.84380003e-14]
Inertia matrix at expressed at the COG  = [1.01377691e-01 6.11265891e-10 2.00884091e-12]
                                          [6.11265891e-10 1.45199268e+00 3.86096397e-09]
                                          [2.00884091e-12 3.86096397e-09 1.45199269e+00]
